In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

sys.path.append("../core")
sys.path.append("../core/simulations")


from losses import getLoss
from trainer import Trainer
from optimizers import getOpt
from network_base import RealNVP
from example_data import gen_double_moon_samples
from network_logging import *
from simulations.visuals import *
from simulations.simulations import *

plt.style.use("ggplot")

## Crescent Moon Example

A basic example of the RealNVP network. The data is generated by mixing samples from two similar distributions such that the shape looks like the following:

<img src="images/doublemoon.png" width=450 height=175>


We want the network to learn how to generate samples from a distribution built from neural network layers. The network starts with being able to generate a simple gaussian distribution and transforms the network parameters so that it generates a more complex distribution.


By running the `crescent_moon_example` we generate that network by training over samples generated from the original distribution. 

In [ ]:
def crescent_moon_example():
    data = gen_double_moon_samples(10000)
    loss = getLoss().maximum_likelihood_loss()
    opt = getOpt().rmsprop(1e-4)
    model = RealNVP(loss, opt, model_name='double_moon')
    model = LogLoss(model)
    model = LogTargetPlot(model, xlim=[-5,5], ylim=[-5,5])
    model = Checkpointing(model)
    trainer = Trainer(model, data)
    trainer.train(20)
    return trainer

In [ ]:
trained = crescent_moon_example()

In [ ]:
samples = trained.model.forward_sample(10000).numpy().T
plt.figure(figsize=(9,6))
plt.title("Samples from Double Moon Trained Model")
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Double Well Example

Now, we try this same method on data from Molecular Dynamics simulations. We are trying to generate samples based on a double well MD simulation that has two potential wells that look like this:

<img src="images/doublewell.png" width=450 height=175>

It is often difficult to start a simulation in one well and reach the other because the potential exponentially climbs in between, so we start first by generating samples from the two wells and then the network from that data.

In [ ]:
def double_well_example():
    sim1 = DoubleWellSim("configs/double_well_config_1.yml")
    sim2 = DoubleWellSim("configs/double_well_config_2.yml")    
    sim1.runSimulation()
    sim2.runSimulation()
    a = np.array(sim1.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    b = np.array(sim2.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    data = np.concatenate((a, b), axis=0)
    
    # we need a function to project our samples for the energy function
    fx = lambda x: x[0]
    loss = getLoss().kl_loss(simulation=sim1)
    opt = getOpt().rmsprop(1e-3)
    model = RealNVP(loss, opt, model_name='double_well')
    model = LogLoss(model)
    model = LogTargetPlot(model, simulation=sim1, xlim=[-3.8,3.8], ylim=[-4.5,4.5])
    model = FreeEnergyPlot(model, simulation=sim1, RC_func=fx)
    model = Checkpointing(model)
    trainer = Trainer(model, data)
    trainer.train(25)
    
    return trainer

In [ ]:
trained_dw = double_well_example()

In [ ]:
samples = trained_dw.model.forward_sample(10000).numpy().T
plt.figure(figsize=(9,6))
plt.title("Samples from Double Moon Trained Model")
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");

## Mueller Potential Well
Now we will try and generate samples from another molecular dynamics simualtion, the mueller potential. This potential has three distinct wells given by the following distribution:

<img src="images/muellerpotential.png" width=450 height=175>

Because of the increased difficulty, we use a different loss function for training.

In [ ]:
def mueller_potential_example():
    sim1 = MuellerWellSim("configs/muller_well_config_1.yml")
    sim2 = MuellerWellSim("configs/muller_well_config_2.yml")
    sim1.runSimulation()
    sim2.runSimulation()
    
    a = np.array(sim1.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    b = np.array(sim2.coordinate_logger.coordinates, dtype=np.float32).squeeze()
    data = np.concatenate((a, b), axis=0)
    
    # a different reaction coordinate projection
    fx = lambda x: np.dot(x, np.array([1, -1]))/np.dot(np.array([1,-1]),np.array([1,-1]))
    
    loss = getLoss().kl_loss(simulation=sim1)
    opt = getOpt().rmsprop(1e-4, clipvalue = 1.0)
    model = RealNVP(loss, opt, model_name="mueller_well", loc=[-0.25, 0.75], scale=[0.5, 0.5])
    model = LogLoss(model)
    model = LogTargetPlot(model, simulation=sim1, xlim=[-5, 5], ylim=[-5, 5])
    model = FreeEnergyPlot(model, simulation=sim1, RC_func=fx)
    trainer = Trainer(model, data)
    trainer.train(25)
    
    return trainer

In [ ]:
trained_mp = mueller_potential_example()

In [ ]:
samples = trained_mp.model.forward_sample(10000).numpy().T
plt.figure(figsize=(9,6))
plt.title("Samples from Mueller Potential Trained Model")
plt.scatter(samples[0], samples[1], s=1.15, c='white', cmap='jet')
plt.xlabel(r"$x_1$")
plt.ylabel(r"$x_2$");